## For TPC-DS dataset with the following query tempalte:
SELECT ss_store_sk, AF(ss_sales_price)

FROM store_sales

WHERE ss_sold_date_sk BETWEEN low AND high

AND ss_coupon_amt=0

GROUP BY ss_store_sk

In [4]:
from dbestclient.executor.executor import SqlExecutor

## Parameter Initialization

In [5]:
sqlExecutor = SqlExecutor()
sqlExecutor.execute("set csv_split_char='|'")
sqlExecutor.execute("set encoder='embedding'")
sqlExecutor.execute("set n_epoch=20")
sqlExecutor.execute(
    "set table_header="
    + "'ss_sold_date_sk|ss_sold_time_sk|ss_item_sk|ss_customer_sk|ss_cdemo_sk|ss_hdemo_sk|"
    + "ss_addr_sk|ss_store_sk|ss_promo_sk|ss_ticket_number|ss_quantity|ss_wholesale_cost|"
    + "ss_list_price|ss_sales_price|ss_ext_discount_amt|ss_ext_sales_price|"
    + "ss_ext_wholesale_cost|ss_ext_list_price|ss_ext_tax|ss_coupon_amt|ss_net_paid|"
    + "ss_net_paid_inc_tax|ss_net_profit|none'"
)

Local mode is on, as no slaves are provided.
start loading pre-existing models.
Loaded 2 models. time cost  0.008768 s
OK, csv_split_char is updated.
OK, encoder is updated.
OK, n_epoch is updated.
OK, table_header is updated.


## Model Creation

In [ ]:
# sqlExecutor.execute("drop table template2")  # drop the model
sqlExecutor.execute(
    "create table template2(ss_sales_price real, ss_sold_date_sk real, ss_coupon_amt categorical) from '/home/quincy/Documents/workspace/data/tpcds/1t/ss_1m.csv' GROUP BY ss_store_sk method uniform size 0.0003472"
)

Start creating model template2
The given table is treated as a uniform sample, and it is obtained with sampling rate 0.000695
Reading data file...
get frequency info from data....
fit MdnQueryEngineXCategoricalOneModel...
training density...
finish training embedding.
< Epoch 0
< Epoch 1
< Epoch 2
< Epoch 3
< Epoch 4
< Epoch 5
< Epoch 6
< Epoch 7
< Epoch 8
< Epoch 9
< Epoch 10
< Epoch 11
< Epoch 12
< Epoch 13
< Epoch 14
< Epoch 15
< Epoch 16
< Epoch 17
< Epoch 18
< Epoch 19
finish mdn training...
training regression...
finish training embedding.
embedding inference...
start normalizing data...
transform data for MDN training...
finish transforming data from MDN training...
< Epoch 0
< Epoch 1
< Epoch 2
< Epoch 3
< Epoch 4
< Epoch 5
< Epoch 6
< Epoch 7
< Epoch 8
< Epoch 9


## Query Serving

In [ ]:
predictions = sqlExecutor.execute(
            "select ss_store_sk, avg(ss_sales_price)  from template2 where   2451119  <=ss_sold_date_sk<= 2451483 and ss_coupon_amt="0" group by ss_store_sk"
        )

# sqlExecutor.execute("drop table template2")

## HIVE query
select ss_store_sk, count(ss_sales_price),sum(ss_sales_price),avg(ss_sales_price)  from store_sales_1t where  ss_sold_date_sk BETWEEN 2451119  AND 2451483  AND ss_coupon_amt=0 group by ss_store_sk